In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
import calendar
import seaborn as sns


%matplotlib inline

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (12, 4)

In [2]:
import sklearn.linear_model as lm

In [3]:
trip_train = pd.read_csv('data/trip_train.csv',low_memory=False);
trip_test = pd.read_csv('data/trip_test.csv',low_memory=False);

In [7]:
trip_x = trip_train[["start_date","start_station_id","subscription_type"]]
trip_y = trip_train["duration"]

In [8]:
def extractYear(s):
    return s.split(' ')[0].split('/')[2]

def extractMonth(s):
    return s.split(' ')[0].split('/')[1]

def extractDay(s):
    return s.split(' ')[0].split('/')[0]

def extractHour(s):
    return s.split(' ')[1].split(':')[0]

def extractMinute(s):
    return s.split(' ')[1].split(':')[1]

trip_x.loc[:,"year"] = trip_x.start_date.apply(extractYear)
trip_x.loc[:,"month"] = trip_x.start_date.apply(extractMonth)
trip_x.loc[:,"day"] = trip_x.start_date.apply(extractDay)
trip_x.loc[:,"hour"] = trip_x.start_date.apply(extractHour)
trip_x.loc[:,"minute"] = trip_x.start_date.apply(extractMinute)
del trip_x["start_date"]
trip_x.head(5)

,start_station_id,subscription_type,year,month,day,hour,minute
0,50,Subscriber,2015,27,8,8,36
1,67,Subscriber,2014,28,7,22,06
2,77,Subscriber,2014,9,6,8,42
3,69,Subscriber,2015,26,1,16,55
4,67,Subscriber,2015,29,8,15,09


In [9]:
# Subscriber -> 1
# Customer -> 0
trip_x.loc[:,"subscription_type"] = trip_x.loc[:,"subscription_type"].apply(lambda x: 1 if x == "Subscriber" else 0)
trip_x.head(5)

,start_station_id,subscription_type,year,month,day,hour,minute
0,50,1,2015,27,8,8,36
1,67,1,2014,28,7,22,06
2,77,1,2014,9,6,8,42
3,69,1,2015,26,1,16,55
4,67,1,2015,29,8,15,09


In [10]:
import sklearn.preprocessing
encoder = sklearn.preprocessing.OneHotEncoder(categorical_features=(0,1,2,3,4))
encoder.fit(trip_x)
encoder.transform(trip_x.head(1)).toarray()

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   8.,  36.]])

In [19]:
encoded_x = pd.DataFrame(encoder.transform(trip_x).toarray())
encoded_x.head(5)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,36.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,22.0,6.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,42.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,55.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,9.0
